In [1]:
# Import dependencies
import os
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '0'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import pandas as pd
import numpy as np
import tensorflow as tf
import logging
from tqdm import tqdm

# Import Tensorflow Keras
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy, BinaryFocalCrossentropy
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Import local modules
from src.utils.consts import TF_RECORD_DATASET, MODELS_PATH, TF_BUFFER_SIZE, NUM_CLASSES, TF_SHUFFLE_SIZE, TF_BATCH_SIZE
from src.model.tensorflow_utils import load_dataset, apply_augmentation_to_dataset, oversample_minority_classes, optimize_dataset, count_dataset_size
from src.model.tensorflow_utils import setup_logger, setup_training_logger, setup_metrics_monitor, setup_loss_monitor, setup_garbage_collector, get_metrics
from src.model.tensorflow_utils import calculate_class_weights, show_class_weights, start_or_resume_training, analyze_class_distribution
from src.model.densnet.tensorflow_dense_net_basic import build_densenet

# Input Data
initial_epoch = 27
resume_training = True
checkpoint_path = '/Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/checkpoints/cp-0026.keras'
model_name = "Simplified_DenseNet_v4"

In [2]:
train_ds = load_dataset(f"{TF_RECORD_DATASET}/train.tfrecord", TF_BUFFER_SIZE)
val_ds   = load_dataset(f"{TF_RECORD_DATASET}/val.tfrecord", TF_BUFFER_SIZE)
test_ds  = load_dataset(f"{TF_RECORD_DATASET}/test.tfrecord", TF_BUFFER_SIZE)

I0000 00:00:1742239340.078612 15058627 pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
I0000 00:00:1742239340.078907 15058627 pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [3]:
# Optimize Dataset for rare clasess
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)
train_ds = oversample_minority_classes(train_ds, class_weights)
class_weights = calculate_class_weights(train_ds, NUM_CLASSES)

In [4]:
steps_per_epoch = int(count_dataset_size(train_ds, None) / TF_BATCH_SIZE)
validation_steps = int(count_dataset_size(val_ds, None) / TF_BATCH_SIZE)

# Testing
train_ds = train_ds.shuffle(TF_SHUFFLE_SIZE, reshuffle_each_iteration=True)
train_ds = apply_augmentation_to_dataset(train_ds)
train_ds = optimize_dataset(train_ds, TF_BATCH_SIZE)

val_ds = optimize_dataset(val_ds, TF_BATCH_SIZE)
test_ds = optimize_dataset(test_ds, TF_BATCH_SIZE)

Counting samples: 108109 samples [1:54:36, 15.72 samples/s]   
Counting samples: 15391 samples [32:06,  7.99 samples/s] 


In [5]:
# Setup Model Deps
# Setup Loggers
logger = setup_logger()
training_logger = setup_training_logger(logger, TF_BATCH_SIZE, 100)
metrics_monitor = setup_metrics_monitor(MODELS_PATH, model_name, logger, resume_training=resume_training, initial_epoch=initial_epoch)
loss_monitor = setup_loss_monitor(MODELS_PATH, model_name, logger, val_ds, resume_training=resume_training, initial_epoch=initial_epoch)
garbage_collector = setup_garbage_collector(logger)
metrics = get_metrics()

# Setup compile arguments
focal_loss  = BinaryFocalCrossentropy(gamma=2.0, alpha=0.25, from_logits=False, label_smoothing=0.01)
reduce_lr = ReduceLROnPlateau(monitor="val_f1_score", factor=0.5,  patience=3, min_lr=1e-6, mode="max", verbose=1)

epoch_mode = 'cp-{epoch:04d}'
save_checkpoint_path = f"{MODELS_PATH}/{model_name}/checkpoints/{epoch_mode}.keras"
checkpoint = ModelCheckpoint(save_checkpoint_path, monitor="val_f1_score", save_best_only=False, mode="max")

model_path = f"{MODELS_PATH}/{model_name}.keras"
best_checkpoint = ModelCheckpoint(model_path, monitor="val_f1_score", save_best_only=True, mode="max")

2025-03-17 22:51:25 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/train_metrics.csv
2025-03-17 22:51:25 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/val_metrics.csv
2025-03-17 22:51:25 - INFO - Cleaned training metrics file, kept 87828 records before epoch 27
2025-03-17 22:51:25 - INFO - Cleaned validation metrics file, kept 26 records before epoch 27
2025-03-17 22:51:25 - INFO - Found 87828 existing training records
2025-03-17 22:51:25 - INFO - Found 26 existing validation records
2025-03-17 22:51:25 - INFO - Cleaned loss analysis metrics file, kept 390 records before epoch 27
2025-03-17 22:51:25 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/loss_analysis_metrics.csv


In [6]:
# Model Training
model = build_densenet(NUM_CLASSES)
compile_kwargs = {'optimizer': Adam(learning_rate=0.0001, clipnorm=1.0), 'loss': focal_loss, 'metrics': metrics}

history, model = start_or_resume_training(
    model, 
    compile_kwargs, 
    train_ds, 
    val_ds, 
    30,
    steps_per_epoch, 
    validation_steps, 
    class_weights=class_weights,
    callbacks=[checkpoint, best_checkpoint, reduce_lr, training_logger, metrics_monitor, loss_monitor, garbage_collector], 
    checkpoint_path=checkpoint_path,
    initial_epoch=initial_epoch,
    output_dir=MODELS_PATH,
    model_name=model_name,
    logger=logger
)

2025-03-17 22:51:26 - INFO - Resuming from existing metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/train_metrics.csv
2025-03-17 22:51:26 - INFO - Resuming from existing validation metrics file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/val_metrics.csv
2025-03-17 22:51:26 - INFO - Cleaned training metrics file, kept 87828 records before epoch 27
2025-03-17 22:51:26 - INFO - Cleaned validation metrics file, kept 26 records before epoch 27
2025-03-17 22:51:26 - INFO - Found 87828 existing training records
2025-03-17 22:51:26 - INFO - Found 26 existing validation records
2025-03-17 22:51:26 - INFO - Cleaned loss analysis metrics file, kept 390 records before epoch 27
2025-03-17 22:51:26 - INFO - Resuming from existing loss analysis file: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/loss_analysis_metrics.csv


Loading full model from checkpoint: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/checkpoints/cp-0026.keras


2025-03-17 22:51:27 - INFO - 
=== Training Started ===

2025-03-17 22:51:27 - INFO - Batch Size: 32
2025-03-17 22:51:27 - INFO - Optimizer: Adam
2025-03-17 22:51:27 - INFO - 

2025-03-17 22:51:27 - INFO - 
=== Starting Epoch 27 ===



Epoch 27/30
3378/3378 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9060 - auc: 0.7068 - f1_score: 0.0429 - loss: 0.0528 - precision: 0.3612 - recall: 0.0289       

2025-03-18 09:21:25 - INFO - 
=== Epoch 27 Summary ===
2025-03-18 09:21:25 - INFO - Time: 37797.60s
2025-03-18 09:21:25 - INFO - Training   - accuracy: 0.9062 - auc: 0.7039 - f1_score: 0.0372 - loss: 0.0534 - precision: 0.3483 - recall: 0.0247 - learning_rate: 0.0001
2025-03-18 09:21:25 - INFO - Validation - accuracy: 0.9125 - auc: 0.6957 - f1_score: 0.0071 - loss: 0.0780 - precision: 0.1570 - recall: 0.0053
2025-03-18 09:21:25 - INFO - ==================================================



...

2025-03-18 10:48:22 - INFO - 
=== Epoch 30 Summary ===
2025-03-18 10:48:22 - INFO - Time: 1781.56s
2025-03-18 10:48:22 - INFO - Training   - accuracy: 0.9065 - auc: 0.7091 - f1_score: 0.0387 - loss: 0.0535 - precision: 0.3713 - recall: 0.0259 - learning_rate: 0.0001
2025-03-18 10:48:22 - INFO - Validation - accuracy: 0.9144 - auc: 0.7088 - f1_score: 0.0067 - loss: 0.0735 - precision: 0.4456 - recall: 0.0044
2025-03-18 10:48:22 - INFO - ==================================================



1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 142ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 105ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 117ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 

2025-03-18 10:49:44 - INFO - 
Loss Analysis - Atelectasis
2025-03-18 10:49:44 - INFO - Confidence Distribution:
2025-03-18 10:49:44 - INFO - -- High (>0.9): 0.00%
2025-03-18 10:49:44 - INFO - -- Medium (0.6-0.9): 0.00%
2025-03-18 10:49:44 - INFO - -- Uncertain (0.4-0.6): 2.05%
2025-03-18 10:49:44 - INFO - -- Low (<0.4): 97.95%
2025-03-18 10:49:44 - INFO - Performance:
2025-03-18 10:49:44 - INFO - -- True Positives: 0
2025-03-18 10:49:44 - INFO - -- False Positives: 0
2025-03-18 10:49:44 - INFO - -- Loss Contribution: 0.0354
2025-03-18 10:49:44 - INFO - Average Confidence:
2025-03-18 10:49:44 - INFO - -- Correct Predictions: 28.70%
2025-03-18 10:49:44 - INFO - -- Incorrect Predictions: 32.48%
2025-03-18 10:49:44 - INFO - 
Loss Analysis - Cardiomegaly
2025-03-18 10:49:44 - INFO - Confidence Distribution:
2025-03-18 10:49:44 - INFO - -- High (>0.9): 0.00%
2025-03-18 10:49:44 - INFO - -- Medium (0.6-0.9): 0.24%
2025-03-18 10:49:44 - INFO - -- Uncertain (0.4-0.6): 4.03%
2025-03-18 10:49:44 

3378/3378 ━━━━━━━━━━━━━━━━━━━━ 1866s 546ms/step - accuracy: 0.9063 - auc: 0.7075 - f1_score: 0.0406 - loss: 0.0533 - precision: 0.3753 - recall: 0.0272 - val_accuracy: 0.9144 - val_auc: 0.7088 - val_f1_score: 0.0067 - val_loss: 0.0735 - val_precision: 0.4456 - val_recall: 0.0044 - learning_rate: 1.0000e-04


2025-03-18 10:49:46 - INFO - 
=== Training Completed! ===

2025-03-18 10:49:46 - INFO - Final Metrics: accuracy: 0.9065 - auc: 0.7091 - f1_score: 0.0387 - loss: 0.0535 - precision: 0.3713 - recall: 0.0259 - val_accuracy: 0.9144 - val_auc: 0.7088 - val_f1_score: 0.0067 - val_loss: 0.0735 - val_precision: 0.4456 - val_recall: 0.0044



In [ ]:
# Initialize the evaluator
from src.model.tensorflow_model_evaluation import ModelEvaluation

mappings_path = f"{TF_RECORD_DATASET}/label_mappings.csv"
evaluator = ModelEvaluation(
    model=model, 
    model_name=model_name, 
    test_dataset=test_ds,
    label_mappings_path=mappings_path,
    output_dir=MODELS_PATH
)

# Cell 2: Generate predictions and basic metrics
metrics = evaluator.evaluate()


Generating predictions...
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 145ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 121ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 120ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 107ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 110ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 108ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 112ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 106ms/step
1

In [ ]:
evaluator.generate_classification_report()


Classification Report:
                    precision    recall  f1-score   support

       Atelectasis       0.00      0.00      0.00      1719
      Cardiomegaly       0.43      0.18      0.25       413
     Consolidation       0.00      0.00      0.00       687
             Edema       0.00      0.00      0.00       342
          Effusion       0.65      0.01      0.01      1984
         Emphysema       0.00      0.00      0.00       376
          Fibrosis       0.00      0.00      0.00       252
            Hernia       0.31      0.24      0.27        34
      Infiltration       0.00      0.00      0.00      2943
              Mass       0.00      0.00      0.00       861
        No Finding       0.00      0.00      0.00      7717
            Nodule       0.00      0.00      0.00       943
Pleural_Thickening       0.00      0.00      0.00       503
         Pneumonia       0.00      0.00      0.00       210
      Pneumothorax       1.00      0.00      0.00       789

         micro

In [ ]:
evaluator.plot_confusion_matrices()


Generating combined confusion matrix...
Saved combined confusion matrix to: /Users/piotr.r/Projects/codebook/studies/bachelor-thesis/models/Simplified_DenseNet_v4/combined_confusion_matrix.png


In [ ]:
for image, labels in test_ds.take(1):
    if len(image.shape) == 4:
        image = image[0]
        labels = labels[0]
    
    evaluator.visualize_prediction(image=image, true_labels=labels)

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 638ms/step
